In [10]:
import pygame
import numpy as np
import enum

pygame.init()

display_width = 800
display_height = 600

game_area_x = 200
game_area_y = 100
game_area_w = 400
game_area_h = 400

game_area_rect = (game_area_x, game_area_y, game_area_w, game_area_h)

white = (255,255,255)
black = (0,0,0)
red = (255,0,0)
green = (0,255,0)
blue = (0,0,255)
yellow = (255,255,0)
dark_red = (200,0,0)
dark_green = (0,200,0)

gameDisplay = pygame.display.set_mode((display_width,display_height))
pygame.display.set_caption('Snake Game')
clock = pygame.time.Clock()

# wrap_allowed = False


def put_score(score):
    font = pygame.font.SysFont(None, 40)
    text = font.render(f"Score: {score}", True, black)
    gameDisplay.blit(text,(0,0))
    
def put_pause_text():
    font = pygame.font.SysFont(None, 25)
    text = font.render("Press P to pause", True, black)
    gameDisplay.blit(text,(0,display_height-30))

class Direction(enum.Enum):
    up = 0
    down = 1
    left = 2
    right = 3
    
dir_speed_dict = {
    Direction.up: (0, -1),
    Direction.down: (0, 1),
    Direction.left: (-1, 0),
    Direction.right: (1,0)
}

key_dir_dict = {
    pygame.K_UP: Direction.up,
    pygame.K_DOWN: Direction.down,
    pygame.K_LEFT: Direction.left,
    pygame.K_RIGHT: Direction.right,
    
}
    
class Snake:
    def __init__(self, block_size=10):
        self.block_size = block_size
        self.points_list = [(np.random.randint(5, game_area_w//block_size-5), 
                             np.random.randint(5, game_area_h//block_size-5))]
        self.direction = Direction(np.random.randint(4))
        
    def get_new_head(self):
        dir_speed = dir_speed_dict[self.direction]
        new_x = self.points_list[0][0] + dir_speed[0]
        new_y = self.points_list[0][1] + dir_speed[1]
        
        wrap_required = False
        if new_x < 0:
            new_x += game_area_w//self.block_size
            wrap_required = True
        elif new_x >= game_area_w//self.block_size:
            new_x -= game_area_w//self.block_size
            wrap_required = True
            
        if new_y < 0:
            new_y += game_area_h//self.block_size
            wrap_required = True
        elif new_y >= game_area_h//self.block_size:
            new_y -= game_area_h//self.block_size
            wrap_required = True
            
        new_head = (new_x, new_y)
        return new_head, wrap_required
    
    def advance(self):
        new_head, wrap_required = self.get_new_head()
        self.points_list = [new_head] + self.points_list[:-1]
    
    def map_snake_point_to_gameDisplay(self, point):
        x,y = point
        return (game_area_x + x*self.block_size, game_area_y + y*self.block_size)
    
    def put_into_display(self):
        # pixAr = pygame.PixelArray(gameDisplay)
#         print("points_list", self.points_list)
        for point in self.points_list:
            # pixAr[x][y] = red
            mapped_point = self.map_snake_point_to_gameDisplay(point)
            x,y = mapped_point
            pygame.draw.rect(gameDisplay, red, (x, y, 
                                                self.block_size, self.block_size))
            
    def turn(self, direction):
        prev_dir = dir_speed_dict[self.direction]
        new_dir = dir_speed_dict[direction]
        dot_prod = prev_dir[0]*new_dir[0] + prev_dir[1]*new_dir[1]
        if dot_prod == 0:
            # change direction
            self.direction = direction
            
    def eat_food(self):
        # collision with food is going to happen in next move
        new_head, wrap_required = self.get_new_head()
        self.points_list = [new_head] + self.points_list
            
            

def put_food(snake, food_point=None):
    if food_point is None:
        food_point = (np.random.randint(game_area_w//snake.block_size), 
                      np.random.randint(game_area_h//snake.block_size))
        while food_point in snake.points_list:
            food_point = (np.random.randint(game_area_w//snake.block_size), 
                          np.random.randint(game_area_h//snake.block_size))
    
    mapped_food_point = snake.map_snake_point_to_gameDisplay(food_point)
    x,y = mapped_food_point
    pygame.draw.rect(gameDisplay, green, (x, y, 
                                                snake.block_size, snake.block_size))
    return food_point
    

def button(msg,x,y,w,h,ic,ac,action=None):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
#     print(click)
    if x+w > mouse[0] > x and y+h > mouse[1] > y:
        pygame.draw.rect(gameDisplay, ac,(x,y,w,h))

        if click[0] == 1 and action != None:
            action()         
    else:
        pygame.draw.rect(gameDisplay, ic,(x,y,w,h))

    smallText = pygame.font.SysFont("comicsansms",20)
    textSurf, textRect = text_objects(msg, smallText)
    textRect.center = ( (x+(w/2)), (y+(h/2)) )
    gameDisplay.blit(textSurf, textRect)   

    
def quitgame():
    pygame.quit()
    
def text_objects(text, font):
    textSurface = font.render(text, True, black)
    return textSurface, textSurface.get_rect()

def game_intro():
    intro = True
    #frame_index = 0
    while intro:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            
            if event.type in [pygame.MOUSEBUTTONDOWN, pygame.MOUSEBUTTONUP]:
                print(event)
                
        #click = pygame.mouse.get_pressed()
        #print(f"game_intro frame {frame_index}: click={click}")
                
        gameDisplay.fill(white)
        largeText = pygame.font.SysFont("comicsansms",115)
        TextSurf, TextRect = text_objects("Snake Game", largeText)
        TextRect.center = ((display_width/2),(display_height/3))
        gameDisplay.blit(TextSurf, TextRect)

        button("GO!",150,450,100,50,green,dark_green,game_level)
        button("Quit",550,450,100,50,red,dark_red,quitgame)

        pygame.display.update()
        #frame_index+=1
        clock.tick(60)
        
def game_level():
    click = pygame.mouse.get_pressed()
    while click[0]==1:
        # while mouse button is pressed
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
#         print(click)
        pygame.display.update()
        clock.tick(30)
        click = pygame.mouse.get_pressed()
    
    intro = True
    while intro:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        # click = pygame.mouse.get_pressed()
        # print(f"Inside game_level: click={click}")
                
        gameDisplay.fill(white)
        largeText = pygame.font.SysFont("comicsansms",50)
        TextSurf, TextRect = text_objects("Choose Level", largeText)
        TextRect.center = ((display_width/2),(display_height/4))
        gameDisplay.blit(TextSurf, TextRect)

        button("Level 1",150,300,100,50,green,dark_green,lambda : game_loop(1))
        button("Level 2",350,300,100,50,green,dark_green,lambda: game_loop(2))
        button("Quit",550,300,100,50,red,dark_red,quitgame)

        pygame.display.update()
        clock.tick(15)

  
def unpause():
    global paused
    paused = False
        
def game_pause():
    global paused
    paused = True
    while paused:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                
        # gameDisplay.fill(white)
        largeText = pygame.font.SysFont("comicsansms",115)
        TextSurf, TextRect = text_objects("Paused", largeText)
        TextRect.center = ((display_width/2),(display_height/2))
        gameDisplay.blit(TextSurf, TextRect)

        button("Resume",150,450,100,50,green,dark_green,unpause)
        button("Quit",550,450,100,50,red,dark_red,quitgame)

        pygame.display.update()
        clock.tick(15)
        
def game_crashed():
    crashed = True
    while crashed:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                
#         gameDisplay.fill(white)
        largeText = pygame.font.SysFont("comicsansms",115)
        TextSurf, TextRect = text_objects("GAME OVER", largeText)
        TextRect.center = ((display_width/2),(display_height/2))
        gameDisplay.blit(TextSurf, TextRect)

        button("Play again",150,450,100,50,green,dark_green,game_level)
        button("Quit",550,450,100,50,red,dark_red,quitgame)

        pygame.display.update()
        clock.tick(15)

def game_loop(level):
    if level==1:
        wrap_allowed = True
    elif level==2:
        wrap_allowed = False
        
    snake = Snake()
    food_point = None
    gameExit = False
    while not gameExit:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                # quit()
 
            if event.type == pygame.KEYDOWN:
                if event.key in [pygame.K_LEFT, pygame.K_RIGHT, pygame.K_UP, pygame.K_DOWN]:
                    # print(f"turn: {key_dir_dict[event.key]}")
                    snake.turn(key_dir_dict[event.key])
                    
                if event.key == pygame.K_p:
                    game_pause()
        
        new_head, wrap_required = snake.get_new_head()
        if not wrap_allowed and wrap_required:
            game_crashed()
            
        if new_head in snake.points_list[:-1]:
            game_crashed()
        
        if  new_head == food_point:
            # collision is going to happen
            snake.eat_food()
            food_point = None
        else:
            snake.advance()
            
        gameDisplay.fill(white)
        pygame.draw.rect(gameDisplay, yellow, game_area_rect)
        put_score(len(snake.points_list)-1)
        put_pause_text()
        snake.put_into_display()
        food_point = put_food(snake, food_point)
        

        pygame.display.update()
        clock.tick(10)

paused = False
game_intro()
 
            
    
        

<Event(5-MouseButtonDown {'pos': (175, 483), 'button': 1, 'window': None})>


error: font not initialized